In [1]:
import pandas as pd
import numpy as np
from binance.client import Client
import time
from binance.client import Client
import pickle
from binance.enums import HistoricalKlinesType
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
import json
import shutil  

Get perpetual pairs

In [2]:
import requests

def get_binance_perpetual_symbols():
    url = "https://fapi.binance.com/fapi/v1/exchangeInfo"
    response = requests.get(url)
    data = response.json()

    perpetual_symbols = [
        symbol_info['symbol']
        for symbol_info in data['symbols']
        if symbol_info['contractType'] == 'PERPETUAL'
    ]
    return perpetual_symbols

symbols = get_binance_perpetual_symbols()
print(symbols)


['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'XMRUSDT', 'DASHUSDT', 'ZECUSDT', 'XTZUSDT', 'BNBUSDT', 'ATOMUSDT', 'ONTUSDT', 'IOTAUSDT', 'BATUSDT', 'VETUSDT', 'NEOUSDT', 'QTUMUSDT', 'IOSTUSDT', 'THETAUSDT', 'ALGOUSDT', 'ZILUSDT', 'KNCUSDT', 'ZRXUSDT', 'COMPUSDT', 'OMGUSDT', 'DOGEUSDT', 'SXPUSDT', 'KAVAUSDT', 'BANDUSDT', 'RLCUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'DEFIUSDT', 'YFIUSDT', 'BALUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT', 'EGLDUSDT', 'SOLUSDT', 'ICXUSDT', 'STORJUSDT', 'BLZUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'ENJUSDT', 'FLMUSDT', 'RENUSDT', 'KSMUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'RSRUSDT', 'LRCUSDT', 'OCEANUSDT', 'BELUSDT', 'AXSUSDT', 'ALPHAUSDT', 'ZENUSDT', 'SKLUSDT', 'GRTUSDT', '1INCHUSDT', 'CHZUSDT', 'SANDUSDT', 'ANKRUSDT', 'LITUSDT', 'UNFIUSDT', 'REEFUSDT', 'RVNUSDT', 'SFPUSDT', 'XEMUSDT', 'COTIUSDT', 'CHRUSDT', 'MANAUSDT', 'ALICEUSDT', 'HBARUSDT', 'ONEUSDT', 'LINAUSDT', '

In [3]:
len(symbols)

518

Reading News File

In [4]:
df_news1 = pd.read_csv('news_20250225to20250510.csv')
df_news2 = pd.read_csv('news_20250510to20250526.csv')
df_news2 = df_news2[~df_news2['ID'].isin(df_news1['ID'])]
df_news = pd.concat([df_news1, df_news2])

df_news['count'] = 1
df_news['releasedAt'] = pd.to_datetime(df_news['releasedAt'])
df_news['releasedAt']

0       2025-02-25
1       2025-02-25
2       2025-02-25
3       2025-02-25
4       2025-02-25
           ...    
68217   2025-05-26
68218   2025-05-26
68219   2025-05-26
68220   2025-05-26
68221   2025-05-26
Name: releasedAt, Length: 309318, dtype: datetime64[ns]

In [5]:
selected_sym = []
list_assets = []
for xx in symbols:
    if('USDT' in xx):
        len_news = len(df_news[df_news['asset_symbols'].astype(str).apply(lambda x: xx[:-4] in x)])
    else:
        continue
    if(len_news>5):
        selected_sym.append(xx)
        list_assets.append({'symb':xx, 'len': len_news})
        # print(xx, len_news)


In [6]:
df_asset = pd.DataFrame(list_assets)
df_asset.sort_values(by='len', ascending=False, inplace=True)
df_asset

,symb,len
309,SUSDT,199659
382,AUSDT,143472
123,TUSDT,139873
307,DUSDT,124009
379,BUSDT,89149
...,...,...
344,FORTHUSDT,6
173,STPTUSDT,6
243,MBOXUSDT,6
134,SSVUSDT,6


Selecting assets with top 100 number of news

In [7]:
list_selected_symb = df_asset.iloc[:100]['symb'].to_list()

Function for Fetching Data from binance api

In [8]:
#Real account
api_key     = ""
api_secret  = ""
client_real = Client(api_key, api_secret)

# Binance API setup
API_KEY = ''  # Replace with your Binance API Key
API_SECRET = ''  # Replace with your Binance Secret Key
client = Client(API_KEY, API_SECRET)


def get_market_data(pair_name, time_frame, start_date, end_date, type_test='offline'):
    klines_hour = client_real.get_historical_klines(pair_name, time_frame, start_date, end_date ,klines_type=HistoricalKlinesType.SPOT)

    df_hour = pd.DataFrame(klines_hour, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])
    df_hour.dateTime = pd.to_datetime(df_hour.dateTime, unit='ms')
    df_hour.closeTime = pd.to_datetime(df_hour.closeTime, unit='ms')
    # df_hour.set_index('dateTime', inplace=True)
    df_hour['close']  = df_hour['close'].apply(lambda x:float(x))
    df_hour['open']   = df_hour['open'].apply(lambda x:float(x))
    df_hour['high']   = df_hour['high'].apply(lambda x:float(x))
    df_hour['low']    = df_hour['low'].apply(lambda x:float(x))
    
    return df_hour

# data = get_market_data('BTCUSDT', '1m', '2025-05-10', '2025-05-27', type_test='offline')

In [ ]:
dict_data = {}
list_data = []
for x in list_selected_symb:
    try:
        data = get_market_data(x, '5m', '2025-05-10', '2025-05-20', type_test='offline')
        data[x] = data['close'].copy()
        list_data.append(data[[x]])
    except Exception as e:
        print(f"Error fetching data for {x}: {e}")
        continue
    # list_data[x]
    # break

Error fetching data for BUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for GUSDT: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error fetching data for LAUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for DEFIUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for SPXUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for TOKENUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for BRUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for BANUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for BANKUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for IPUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for PUMPUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for ARCUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for HYPEUSDT: APIError(code=-1121): Invalid symbol.
Error fetching data for

In [ ]:
df_all = pd.concat(list_data, axis=1)
df_all[list((df_all.isna().sum()>0).index)]

,SUSDT,AUSDT,TUSDT,DUSDT,WUSDT,BTCUSDT,AIUSDT,MEUSDT,ETHUSDT,OMUSDT,...,TIAUSDT,JSTUSDT,VETUSDT,FILUSDT,ACHUSDT,ENAUSDT,ZILUSDT,FORMUSDT,THEUSDT,BIOUSDT
0,0.5921,NaN,0.01882,0.05124,0.1003,103092.49,0.1926,1.116,2353.17,0.3858,...,3.064,0.03844,0.03013,3.033,0.02931,0.3695,0.01418,2.3436,0.3587,0.0915
1,0.5950,NaN,0.01890,0.05175,0.1012,103077.49,0.1944,1.105,2353.00,0.3857,...,3.084,0.03830,0.03026,3.043,0.02938,0.3722,0.01424,2.3543,0.3683,0.0902
2,0.5957,NaN,0.01890,0.05190,0.1019,102999.99,0.1963,1.110,2349.17,0.3859,...,3.096,0.03817,0.03028,3.040,0.02945,0.3731,0.01430,2.3514,0.3733,0.0917
3,0.5982,NaN,0.01911,0.05197,0.1030,103039.20,0.2008,1.121,2367.40,0.3876,...,3.126,0.03825,0.03057,3.060,0.02987,0.3754,0.01437,2.3433,0.3807,0.0925
4,0.5990,NaN,0.01909,0.05171,0.1036,102956.67,0.1989,1.119,2371.82,0.3898,...,3.125,0.03827,0.03049,3.068,0.02977,0.3731,0.01437,2.3575,0.3699,0.0912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2876,0.4989,NaN,0.01727,0.04262,0.0943,105603.53,0.1753,0.952,2530.56,0.3900,...,2.634,0.03523,0.02802,2.856,0.02494,0.3674,0.01263,2.6132,0.3032,0.0739
2877,0.4997,NaN,0.01729,0.04270,0.0944,105599.99,0.1757,0.954,2530.69,0.3897,...,2.638,0.03525,0.02805,2.860,0.02496,0.3697,0.01264,2.6075,0.3033,0.0741
2878,0.4996,NaN,0.01728,0.04275,0.0945,105600.00,0.1760,0.955,2528.60,0.3895,...,2.640,0.03522,0.02807,2.863,0.02494,0.3703,0.01261,2.6087,0.3037,0.0741
2879,0.4997,NaN,0.01727,0.04271,0.0944,105573.74,0.1757,0.953,2528.14,0.3898,...,2.636,0.03522,0.02804,2.859,0.02489,0.3696,0.01262,2.6076,0.3032,0.0740


In [ ]:
df_asset_no_nan = df_all.dropna(axis=1)
df_corr = df_asset_no_nan.corr()
df_corr[['BTCUSDT']].sort_values(by='BTCUSDT', ascending=False)

,BTCUSDT
BTCUSDT,1.000000
BBUSDT,0.611883
AAVEUSDT,0.471582
ALPHAUSDT,0.429499
DOGEUSDT,0.385485
...,...
TRXUSDT,-0.076011
OGUSDT,-0.081079
XRPUSDT,-0.095728
FORMUSDT,-0.142171


In [ ]:
list_stocks = df_corr[['BTCUSDT']].sort_values(by='BTCUSDT', ascending=True).index.to_list()